In [1]:
import json
from copy import deepcopy
from importlib import import_module
from pathlib import Path

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

seed = 1
knob_types = ['enum', 'integer', 'real']
valid_adapter_aliases = ['none', 'tree', 'rembo', 'hesbo']

In [2]:
include = None
ignore = []

pgsql_knob_json = "/home/viktor/Experiments/TPC-H/knob_definitions/postgres-13.json"
with open(pgsql_knob_json, 'r') as f:
    definitions = json.load(f)

all_knobs = set([d['name'] for d in  definitions])
include_knobs = include if include is not None else all_knobs - set(ignore) 

knobs = [info for info in definitions if info['name'] in include_knobs]
knobs_dict = { d['name']: d for d in knobs}

In [3]:
input_dimensions = [ ]
for info in knobs:
    name, knob_type = info['name'], info['type']
    
    if knob_type not in knob_types:
        raise NotImplementedError(f'Knob type of "{knob_type}" is not supported :(')

    ## Categorical
    if knob_type == 'enum':
        dim = CSH.CategoricalHyperparameter(
                        name=name,
                        choices=info['choices'],
                        default_value=info['default'])
    ## Numerical
    elif knob_type == 'integer':
        dim = CSH.UniformIntegerHyperparameter(
                        name=name,
                        lower=info['min'],
                        upper=info['max'],
                        default_value=info['default'])
    elif knob_type == 'real':
        dim = CSH.UniformFloatHyperparameter(
                        name=name,
                        lower=info['min'],
                        upper=info['max'],
                        default_value=info['default'])
    
    input_dimensions.append(dim)

input_space = CS.ConfigurationSpace(name='input', seed=seed)
input_space.add_hyperparameters(input_dimensions)

[autovacuum, Type: Categorical, Choices: {on, off}, Default: on,
 autovacuum_analyze_scale_factor, Type: UniformFloat, Range: [0.0, 100.0], Default: 0.1,
 autovacuum_analyze_threshold, Type: UniformInteger, Range: [0, 1000], Default: 50,
 autovacuum_freeze_max_age, Type: UniformInteger, Range: [100000, 2000000000], Default: 200000000,
 autovacuum_max_workers, Type: UniformInteger, Range: [1, 10000], Default: 3,
 autovacuum_multixact_freeze_max_age, Type: UniformInteger, Range: [10000, 2000000000], Default: 400000000,
 autovacuum_naptime, Type: UniformInteger, Range: [1, 2147483], Default: 60,
 autovacuum_vacuum_cost_delay, Type: UniformInteger, Range: [-1, 100], Default: 2,
 autovacuum_vacuum_cost_limit, Type: UniformInteger, Range: [-1, 10000], Default: -1,
 autovacuum_vacuum_insert_scale_factor, Type: UniformFloat, Range: [0.0, 100.0], Default: 0.2,
 autovacuum_vacuum_insert_threshold, Type: UniformInteger, Range: [-1, 100000], Default: 1000,
 autovacuum_vacuum_scale_factor, Type: Un

In [4]:
sample_config = input_space.sample_configuration()
sample_config

Configuration(values={
  'autovacuum': 'on',
  'autovacuum_analyze_scale_factor': 72.0324493442158,
  'autovacuum_analyze_threshold': 0,
  'autovacuum_freeze_max_age': 604734912,
  'autovacuum_max_workers': 1468,
  'autovacuum_multixact_freeze_max_age': 184686266,
  'autovacuum_naptime': 399991,
  'autovacuum_vacuum_cost_delay': 34,
  'autovacuum_vacuum_cost_limit': 3967,
  'autovacuum_vacuum_insert_scale_factor': 53.88167340033569,
  'autovacuum_vacuum_insert_threshold': 41919,
  'autovacuum_vacuum_scale_factor': 68.52195003967594,
  'autovacuum_vacuum_threshold': 439057863,
  'autovacuum_work_mem': 920773,
  'backend_flush_after': 7,
  'bgwriter_delay': 2685,
  'bgwriter_flush_after': 107,
  'bgwriter_lru_maxpages': 559,
  'bgwriter_lru_multiplier': 1.1230955087618701,
  'checkpoint_completion_target': 0.1981014890848788,
  'checkpoint_flush_after': 205,
  'checkpoint_timeout': 83659,
  'commit_delay': 1284,
  'commit_siblings': 11,
  'cpu_index_tuple_cost': 3.5055566091841532,
  'cp

In [ ]:
knobs = sample_config.get_dictionary().copy()